# Tech Intro

## Main tables

In [597]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [598]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[0] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [599]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 00:43:58.758584 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 00:43:58.759461 44788 sched.cpp:241] Version: 1.11.0
I1120 00:43:58.759711 10542 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 00:43:58.759789 10542 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 00:43:58.761188 10557 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75605


## Read the spec-file from Hadoop and show some data sample

In [600]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+------+----+----------------------+--------+----------+-------+----+---------+-------+-------+----------+----------+---------+---------+--------+---------+----------+-----------+----+-------+
|user_id|id    |type|object_type           |reg     |updated   |deleted|arch|arch_year|jurist |rieltor|start_deal|end_deal  |documents|parent_id|leads_id|filial_id|roistat_id|status     |uss |uss_ruk|
+-------+------+----+----------------------+--------+----------+-------+----+---------+-------+-------+----------+----------+---------+---------+--------+---------+----------+-----------+----+-------+
|12942  |100002|5   |Вторичная недвижимость|42333455|1580993679|0      |859 |20       |null   |3359901|1580947200|null      |null     |null     |null    |1        |null      |Действующий|null|null   |
|206    |10001 |6   |null                  |13312138|1526369586|0      |3708|18       |206    |290    |1526369586|null      |null     |null     |null    |1        |null      |Действующий|null|null

## Show columns' properties

In [601]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- user_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- object_type: string (nullable = true)
 |-- reg: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- arch: string (nullable = true)
 |-- arch_year: string (nullable = true)
 |-- jurist: string (nullable = true)
 |-- rieltor: string (nullable = true)
 |-- start_deal: string (nullable = true)
 |-- end_deal: string (nullable = true)
 |-- documents: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- leads_id: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- roistat_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- uss: string (nullable = true)
 |-- uss_ruk: string (nullable = true)



## Total rows (count)

In [602]:
eda_df.count()

196714

## Simple func to execute and show the result of the SQL-instruction

In [603]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [604]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [605]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|196714  |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [606]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,user_id,id,type,object_type,reg,updated,deleted,arch,arch_year,jurist,...,start_deal,end_deal,documents,parent_id,leads_id,filial_id,roistat_id,status,uss,uss_ruk
0,0,0,15833,38800,1200,6088,0,0,0,50477,...,6092,136906,190762,195225,195172,5928,196714,6473,182934,196439


In [607]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,user_id,id,type,object_type,reg,updated,deleted,arch,arch_year,jurist,...,start_deal,end_deal,documents,parent_id,leads_id,filial_id,roistat_id,status,uss,uss_ruk
0,1200,0,0,0,0,0,188198,43251,43251,0,...,0,0,0,0,0,0,0,0,0,0


In [608]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,user_id,id,type,object_type,reg,updated,deleted,arch,arch_year,jurist,...,start_deal,end_deal,documents,parent_id,leads_id,filial_id,roistat_id,status,uss,uss_ruk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2327,0,0


In [609]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
user_id,0,1200,0
id,0,0,0
type,15833,0,0
object_type,38800,0,0
reg,1200,0,0
updated,6088,0,0
deleted,0,188198,0
arch,0,43251,0
arch_year,0,43251,0
jurist,50477,0,0


In [610]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [611]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
user_id,0.000,0.610,0.000
id,0.000,0.000,0.000
type,8.049,0.000,0.000
object_type,19.724,0.000,0.000
reg,0.610,0.000,0.000
updated,3.095,0.000,0.000
deleted,0.000,95.671,0.000
arch,0.000,21.987,0.000
arch_year,0.000,21.987,0.000
jurist,25.660,0.000,0.000


### SQL-instructions

In [612]:
#Find dupes or misunderstanding or useless static values 196 714 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|936                    |
+-----------------------+

+------------------+
|count(DISTINCT id)|
+------------------+
|196714            |
+------------------+

+--------------------+
|count(DISTINCT type)|
+--------------------+
|6                   |
+--------------------+

+---------------------------+
|count(DISTINCT object_type)|
+---------------------------+
|5                          |
+---------------------------+

+-------------------+
|count(DISTINCT reg)|
+-------------------+
|195448             |
+-------------------+

+-----------------------+
|count(DISTINCT updated)|
+-----------------------+
|190225                 |
+-----------------------+

+-----------------------+
|count(DISTINCT deleted)|
+-----------------------+
|2                      |
+-----------------------+

+--------------------+
|count(DISTINCT arch)|
+--------------------+
|11816               |
+--------------------+

+------

In [613]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [614]:
#Find dupes or misunderstanding or useless static values 196 714 total
for col in eda_df.columns:
    execute_template(col)

+-------+
|user_id|
+-------+
|9679   |
|9677   |
|9672   |
|9626   |
|9625   |
|9621   |
|9582   |
|9581   |
|9579   |
|908    |
|903    |
|898    |
|897    |
|895    |
|872    |
|871    |
|870    |
|863    |
|8480   |
|8473   |
+-------+
only showing top 20 rows

None
+-------+
|user_id|
+-------+
|0      |
|10188  |
|10190  |
|10225  |
|10739  |
|11258  |
|11260  |
|11312  |
|11350  |
|11390  |
|11397  |
|11401  |
|11403  |
|11404  |
|11470  |
|11487  |
|11497  |
|11499  |
|11500  |
|11505  |
+-------+
only showing top 20 rows

+-----+
|id   |
+-----+
|99999|
|99998|
|99997|
|99996|
|99995|
|99994|
|99993|
|99992|
|99991|
|99990|
|9999 |
|99989|
|99988|
|99987|
|99986|
|99985|
|99984|
|99983|
|99982|
|99981|
+-----+
only showing top 20 rows

None
+------+
|id    |
+------+
|100   |
|1000  |
|10000 |
|100000|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|10001 |
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
+------+
only showing top 20 rows

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [596]:
spark.stop()

I1120 00:43:52.901058 10390 sched.cpp:2174] Asked to stop the driver
I1120 00:43:52.901201 10546 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75604
